In [1]:
import numpy as np
import torch
from torch import nn
from tropical import Tropical, TropicalMonomial, TropicalPolynomial
from tropical import to_tensor, Tropical, TropicalMonomial, TropicalPolynomial, PolyNet, DiffPolyNet

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def to_tensor(x):
    return torch.tensor(x).float()

In [4]:
t = Tropical(0)

In [5]:
a = TropicalPolynomial([[0,1,0],[0,0,1]])

In [6]:
a

a ⨁ b

In [8]:
x = [-100,-2]

In [9]:
a.evaluate(x)

-2

In [10]:
a_net = PolyNet(a)

In [11]:
a_net

PolyNet(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [12]:
a_net.forward(torch.tensor(x).float())

tensor([-2.], grad_fn=<AddBackward0>)

In [14]:
def convert_net_to_tropical(net):
    
    d = net.net[0].in_features
    f = [TropicalPolynomial([[0]+np.eye(d)[i].tolist()]) for i in range(d)]
    
    g = [TropicalPolynomial([[0]+np.zeros(d).tolist()]) for i in range(d)]


    for l in net.net[::2]:
        
        n = l.in_features
        m = l.out_features
        a = l.weight.data.detach().cpu().numpy()
        a_plus = np.maximum(a,0)
        a_minus = np.maximum(-a,0)
        b_ = l.bias.data.detach().cpu().numpy()
        
        new_g = []
        new_h = []
        new_f = []
        
        for i in range(m):
            g_i = None
            h_i = None
            f_i = None
            for j in range(n):
                if g_i is None:
                    g_i = f[j]**a_minus[i][j]
                    g_i *= g[j]**a_plus[i][j]
                else:
                    g_i *= f[j]**a_minus[i][j]
                    g_i *= g[j]**a_plus[i][j]
                
                if h_i is None:
                    h_i = f[j]**a_plus[i][j]
                    h_i *= g[j]**a_minus[i][j]
                else:
                    h_i *= f[j]**a_plus[i][j]
                    h_i *= g[j]**a_minus[i][j]
                
            h_i *= Tropical(b_[i])
            f_i = h_i+g_i*t
            
            new_g.append(g_i)
            new_h.append(h_i)
            new_f.append(f_i)
        
        f = new_f
        g = new_g
        h = new_h
        
    if isinstance(net.net[-1], nn.Linear):
        return h,g
    else:
        return f,g

In [15]:
f,g = convert_net_to_tropical(a_net)

In [16]:
f

[a⨀b^2.0 ⨁ b^3.0 ⨁ a⨀b ⨁ b^2.0]

In [17]:
g

[b ⨁ b^2.0]

In [19]:
f[0].evaluate(x) / g[0].evaluate(x)

-2.0

In [51]:
f[0]

a⨀b^2.0 ⨁ b^3.0 ⨁ a⨀b ⨁ b^2.0

In [52]:
f[0].minimize()

a⨀b^2.0 ⨁ b^3.0 ⨁ b^2.0 ⨁ a⨀b

In [65]:
a = TropicalPolynomial([[1,2,0],[1,0,2],[1,1,1]])

In [66]:
a

1⨀a^2 ⨁ 1⨀b^2 ⨁ 1⨀a⨀b

In [67]:
a.minimize()

1⨀a^2 ⨁ 1⨀b^2